In [1]:
def _get_block_length(n):
    bits = n.bit_length()
    m = (bits - 1) // 64
    n = m * 8
    
    return n

def _get_encrypted_block_length(n):
    bits = n.bit_length()
    return math.ceil(bits/8)

    

In [2]:
from sympy import randprime
import math
import random

def find_relative_prime(n):
    while True:
        candidate = random.randint(1, n-1)
        if math.gcd(candidate, n) == 1:
            return candidate

class RSAReceiver:
    def __init__(self, bit_len):
        self.bit_len = bit_len
        self._generate_keys()

    def get_public_key(self):
        return self.public_key

    def decrypt(self, ciphertext):
        array = ciphertext
        n = _get_encrypted_block_length(self.secret_key[1])
        
        
        decoded_array = bytearray()
        for i in range(0, len(array), n):
            block = array[i: i+n]
            decoded_array += self._decode_block(block)
        return decoded_array.decode('UTF-8')
    
    def _generate_keys(self):
        p = randprime(2 ** self.bit_len, 2 ** (self.bit_len + 1))
        while True:
            q = randprime(2 ** self.bit_len, 2 ** (self.bit_len + 1))
            if q != p:
                break
        n = p * q
        
        phi = (p - 1)*(q - 1)
        
        d = find_relative_prime(phi)
        e = pow(d, -1, phi)
        
        self.public_key = (e, n)
        self.secret_key = (d, n)

    def _decode_block(self, block):
        coded_number = int.from_bytes(block, 'big')
        decoded_number = pow(coded_number, self.secret_key[0], self.secret_key[1])
        desincrypted_block = decoded_number.to_bytes(len(block), 'big')
        return desincrypted_block
        

In [5]:
class RSASender:
    def __init__(self, public_key, secret_key):
        self.public_key = public_key
        self.secret_key = secret_key
    
    def encrypt(self, message):  
        n = _get_block_length(self.public_key[1])
        array = bytearray(message, 'UTF-8')

        encoded_array = bytearray()
        for i in range(0, len(array), n):
            right_limit = min(len(array), i + n)
            block = array[i: right_limit]

            encrypted_block = self._encrypt_block(block)
            encoded_array += encrypted_block
        
        return encoded_array
    
    def _encrypt_block(self, block):
        number = int.from_bytes(block, 'big')
        encoded_number = pow(number, self.public_key[0], self.public_key[1])
        encoded_block_len = _get_encrypted_block_length(self.public_key[1])
        encoded_block = encoded_number.to_bytes(encoded_block_len, 'big')
        return encoded_block

In [4]:
#receiver = RSAReceiver(100)
#pk = receiver.get_public_key()
#sender = RSASender(pk, receiver.secret_key)

#text = '1aasawedf' * 10

#encrypted = sender.encrypt(text)
#decrypted = receiver.decrypt(encrypted)
#print(decrypted)